# 📊 Conformal Interval Calibration

This notebook demonstrates how to wrap a `RandomForestQuantileRegressor` in MAPIE's conformal quantile regression (CQR) framework and evaluate calibration in a rolling-window backtest.


In [ ]:
import pandas as pd
import numpy as np
from quantile_forest import RandomForestQuantileRegressor
from mapie.regression import MapieQuantileRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt

# Assume df_model is already loaded with feature columns and 'regime' flag
# features = [...]  # your 12h feature columns
# target = 'return_12h'


In [ ]:
window_train = 120
window_cal = 30
window_test = 30

records = []
for start in range(0, len(df_model) - (window_train + window_cal + window_test), window_test):
    train = df_model.iloc[start : start + window_train]
    cal = df_model.iloc[start + window_train : start + window_train + window_cal]
    test = df_model.iloc[start + window_train + window_cal : start + window_train + window_cal + window_test]
    X_train, y_train = train[features], train[target]
    X_cal, y_cal = cal[features], cal[target]
    X_test, y_test = test[features], test[target]

    qrf = RandomForestQuantileRegressor(random_state=0)
    qrf.fit(X_train, y_train)
    cqr = MapieQuantileRegressor(estimator=qrf, method="base+conformity")
    cqr.fit(pd.concat([X_train, X_cal]), pd.concat([y_train, y_cal]))

    lower, upper = cqr.predict(X_test, alpha=0.2)
    delta = cqr.delta_
    emp_cov = ((y_test >= lower) & (y_test <= upper)).mean()
    width = (upper - lower).mean()
    median_pred = qrf.predict(X_test, quantile=50)
    residuals = y_test - median_pred
    records.append({"start": start, "delta": delta, "emp_cov": emp_cov, "width": width, "regime": test['regime'].mode()[0], "residuals": residuals})


In [ ]:
results = pd.DataFrame(records)

# Overall calibration
nominal = 0.8
empirical = results['emp_cov'].mean()
print(f"Nominal {nominal:.1%} -> Empirical {empirical:.1%}")

# Coverage by regime
by_regime = results.groupby('regime')[['emp_cov','width']].mean()
print(by_regime)

# Plot delta over time
plt.figure()
plt.plot(results['start'], results['delta'])
plt.title('Conformal delta over time')
plt.xlabel('Window start index')
plt.ylabel('delta')
plt.show()

# Residual ACF
all_resid = np.concatenate([r for r in results['residuals']])
plot_acf(all_resid, lags=20)
plt.show()


## Feature-wise coverage analysis
Example of binning a feature and computing coverage in each bin.


In [ ]:
feature_bins = pd.qcut(df_model['volatility_72h'], q=5)
coverage_by_bin = pd.DataFrame({
    'bin': feature_bins,
    'covered': ((df_model[target] >= lower) & (df_model[target] <= upper))
})
heatmap = coverage_by_bin.groupby('bin')['covered'].mean()
print(heatmap)


This notebook outlines the rolling conformal interval calibration approach. Replace placeholder code with your data loading and feature definitions.